# PyOPE vs Mathematica OPEdefs 对比测试

本 notebook 用于对比 PyOPE 和 Mathematica OPEdefs.m 的计算结果，验证 Python 实现的正确性。

## 测试策略

1. 使用 `wolframscript` 运行 Mathematica 代码
2. 使用 PyOPE 运行相同的计算
3. 对比两者的结果

In [ ]:
import subprocess
import json
import sympy as sp
from pathlib import Path

# 导入 PyOPE
from pyope import (
    BasisOperator,
    d,
    One,
    OPE,
    NO,
    bracket,
    MakeOPE,
    Bosonic,
    Fermionic,
)

print("✓ 模块导入成功")

## 辅助函数：运行 Mathematica 代码

In [ ]:
def run_mathematica(code: str) -> str:
    """
    使用 wolframscript 运行 Mathematica 代码
    
    Args:
        code: Mathematica 代码字符串
    
    Returns:
        输出结果字符串
    """
    try:
        result = subprocess.run(
            ['wolframscript', '-code', code],
            capture_output=True,
            text=True,
            timeout=30
        )
        if result.returncode != 0:
            print(f"错误: {result.stderr}")
            return None
        return result.stdout.strip()
    except subprocess.TimeoutExpired:
        print("Mathematica 执行超时")
        return None
    except Exception as e:
        print(f"执行出错: {e}")
        return None

# 测试 wolframscript
test_result = run_mathematica("2 + 2")
print(f"测试 Mathematica: 2 + 2 = {test_result}")

## 测试 1: 基本 OPE 定义和计算

测试 Virasoro 代数的 OPE: T(z)T(w)

In [ ]:
# Mathematica 代码
mathematica_code = """
Get["/Users/lelouch/pyope/OPEdefs/OPEdefs.m"];
Bosonic[T];
OPE[T, T] = MakeOPE[{c/2 One, 0, 2 T, T'}];
result = OPE[T, T];
Print[result]
"""

print("=== Mathematica 结果 ===")
mathematica_result = run_mathematica(mathematica_code)
print(mathematica_result)
print()

# PyOPE 代码
print("=== PyOPE 结果 ===")
T = BasisOperator("T", bosonic=True, conformal_weight=2)
Bosonic(T)
c = sp.Symbol('c')

OPE[T, T] = MakeOPE([c/2 * One, 0, 2*T, d(T)])
pyope_result = OPE(T, T)

print(f"最高极点: {pyope_result.max_pole}")
for q in range(pyope_result.max_pole, 0, -1):
    pole_q = pyope_result.pole(q)
    if pole_q != 0:
        print(f"  pole({q}): {pole_q}")

## 测试 2: 正规序乘积

测试 NO(T, T) 的计算

In [ ]:
# Mathematica 代码
mathematica_code = """
Get["/Users/lelouch/pyope/OPEdefs/OPEdefs.m"];
Bosonic[T];
result = NO[T, T];
Print[result]
"""

print("=== Mathematica 结果 ===")
mathematica_result = run_mathematica(mathematica_code)
print(mathematica_result)
print()

# PyOPE 代码
print("=== PyOPE 结果 ===")
no_TT = NO(T, T)
print(f"NO(T, T) = {no_TT}")

## 测试 3: 左侧复合算符的 OPE

测试 OPE(NO(J,J), J) - 这是我们刚刚完善的功能

In [ ]:
# Mathematica 代码
mathematica_code = """
Get["/Users/lelouch/pyope/OPEdefs/OPEdefs.m"];
Bosonic[J];
OPE[J, J] = MakeOPE[{k One, 0}];
result = OPE[NO[J, J], J];
Print["MaxPole: ", MaxPole[result]];
Print["Pole 2: ", OPEPole[2][result]];
Print["Pole 1: ", OPEPole[1][result]];
"""

print("=== Mathematica 结果 ===")
mathematica_result = run_mathematica(mathematica_code)
print(mathematica_result)
print()

# PyOPE 代码
print("=== PyOPE 结果 ===")
J = BasisOperator("J", bosonic=True, conformal_weight=1)
Bosonic(J)
k = sp.Symbol('k')

OPE[J, J] = MakeOPE([k * One, 0])
no_JJ = NO(J, J)
pyope_result = OPE(no_JJ, J)

print(f"MaxPole: {pyope_result.max_pole}")
print(f"Pole 2: {pyope_result.pole(2)}")
print(f"Pole 1: {pyope_result.pole(1)}")

## 测试 4: 导数算符的 OPE

测试 OPE(∂T, T)

In [ ]:
# Mathematica 代码
mathematica_code = """
Get["/Users/lelouch/pyope/OPEdefs/OPEdefs.m"];
Bosonic[T];
OPE[T, T] = MakeOPE[{c/2 One, 0, 2 T, T'}];
result = OPE[T', T];
Print["MaxPole: ", MaxPole[result]];
Do[Print["Pole ", i, ": ", OPEPole[i][result]], {i, MaxPole[result], 1, -1}];
"""

print("=== Mathematica 结果 ===")
mathematica_result = run_mathematica(mathematica_code)
print(mathematica_result)
print()

# PyOPE 代码
print("=== PyOPE 结果 ===")
dT = d(T)
pyope_result = OPE(dT, T)

print(f"MaxPole: {pyope_result.max_pole}")
for q in range(pyope_result.max_pole, 0, -1):
    pole_q = pyope_result.pole(q)
    if pole_q != 0:
        print(f"Pole {q}: {pole_q}")

## 测试 5: 混合算符的左侧复合 OPE

测试 OPE(NO(T,J), J)

In [ ]:
# Mathematica 代码
mathematica_code = """
Get["/Users/lelouch/pyope/OPEdefs/OPEdefs.m"];
Bosonic[T, J];
OPE[T, T] = MakeOPE[{c/2 One, 0, 2 T, T'}];
OPE[J, J] = MakeOPE[{k One, 0}];
OPE[T, J] = MakeOPE[{J, J'}];
result = OPE[NO[T, J], J];
Print["MaxPole: ", MaxPole[result]];
Print["Pole 2: ", OPEPole[2][result]];
Print["Pole 1: ", OPEPole[1][result]];
"""

print("=== Mathematica 结果 ===")
mathematica_result = run_mathematica(mathematica_code)
print(mathematica_result)
print()

# PyOPE 代码
print("=== PyOPE 结果 ===")
# 重新定义算符（避免之前的定义干扰）
T2 = BasisOperator("T", bosonic=True, conformal_weight=2)
J2 = BasisOperator("J", bosonic=True, conformal_weight=1)
Bosonic(T2, J2)

OPE[T2, T2] = MakeOPE([c/2 * One, 0, 2*T2, d(T2)])
OPE[J2, J2] = MakeOPE([k * One, 0])
OPE[T2, J2] = MakeOPE([J2, d(J2)])

no_TJ = NO(T2, J2)
pyope_result = OPE(no_TJ, J2)

print(f"MaxPole: {pyope_result.max_pole}")
print(f"Pole 2: {pyope_result.pole(2)}")
print(f"Pole 1: {pyope_result.pole(1)}")

## 测试总结

运行上述所有测试，对比 Mathematica 和 PyOPE 的结果。

In [ ]:
print("="*60)
print("测试完成！")
print("="*60)
print()
print("请检查上述各个测试的结果，确认 PyOPE 与 Mathematica 的输出是否一致。")
print()
print("注意事项：")
print("1. Mathematica 和 Python 的输出格式可能不同")
print("2. 符号表示可能有差异（如 T' vs ∂T）")
print("3. 重点关注数学结构和系数是否相同")